In [1]:
from catboost import CatBoostClassifier, Pool, cv
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from xgboost import XGBClassifier
from train_val_split import train_validation_split
import ggplot as gplt
from tools.mean_encoder import *
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from autoencoders_keras.vanilla_autoencoder import VanillaAutoencoder

C:\Users\S\Anaconda3\lib\site-packages\ggplot\utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
C:\Users\S\Anaconda3\lib\site-packages\ggplot\stats\smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
C:\Users\S\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\S\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import r

In [28]:
train_set = pd.read_csv("dataset/train_val/training_set.csv")
test_set = pd.read_csv("dataset/train_val/testing_set.csv")
train_set["order_id"] = train_set["order_id"].astype(str)

In [2]:
train_set = pd.read_csv("dataset/train_val/training_set.csv")
test_set = pd.read_csv("dataset/train_val/testing_set.csv")
train_set["order_id"] = train_set["order_id"].astype(str)


airport_me = pd.read_csv("./feature_extraction/airport_mean_encoding.csv")
airport_me["order_id"] = airport_me["order_id"].astype(str)
train_set = pd.merge(train_set, airport_me, on="order_id", how="left")
test_set = pd.merge(test_set, airport_me, on="order_id", how="left")


drop_col = [x for x in train_set.columns if "time_dist" in x]

y_train = train_set["deal_or_not"]
for table in [train_set, test_set]:
    table.drop(columns=["deal_or_not", "group_id", "order_id"] + drop_col, inplace=True)
    table.fillna(table.mean(), inplace=True)
    

cv = StratifiedKFold(5, shuffle=True, random_state=851206)
#mean_enc_col = ["source1_unit", "area", 'source1_source2', 'source1_source2_unit',
#                'source2_unit'] # 這個column是catboost feature importance最高的feature
mean_enc_col = ["source1_unit"]
#mean_enc_col = list(train_set.columns[cat_feature])
train_set = mean_encoder(train_set, y_train, mean_enc_col, "deal_or_not", cv)
test_set = test_set_encoder(train_set, y_train, test_set, mean_enc_col, "deal_or_not")

drop_col = ['source1_source2', 'source1_unit', 'source2_unit',
       'source1_source2_unit', 'order_month', 'order_quarter', 'subline_area',
       'begin_month', 'begin_quarter', 'abroad_airport', 'home_airport',
       'abroad_hour', 'abroad_part_of_day', 'abroad_DoY', 'abroad_DoW',
       'abroad_DoM', 'home_DoW', 'home_DoM', 'home_DoY', 'home_hour',
       'home_part_of_day']
for table in [train_set, test_set]:
    table.drop(columns=drop_col, inplace=True)

#and "DoY" not in col
cat_feature = []
key_words = ["source", "unit", "sub", "area", "order", "begin", "_airport",
             "abroad", "home"]
for i, col in enumerate(train_set.columns):
    for kw in key_words:
        if kw in col and "duration" not in col and "DoY" not in col and "target" not in col \
         and "accum" not in col and i not in cat_feature:
            cat_feature.append(i)

            
#train_x.drop(columns=mean_enc_col, inplace=True)
#val_x.drop(columns=mean_enc_col, inplace=True)

C:\Users\S\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\S\travel_data\tools\mean_encoder.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  x_val[colname] = means


In [3]:
whole_set = pd.concat([train_set, test_set])
whole_set.index = list(range(whole_set.shape[0]))

In [4]:
def dummy_rep(table, cat_feature):
    dummy = pd.get_dummies(table[cat_feature[0]])
    for col in cat_feature[1:]:
        dummy = pd.concat([dummy, pd.get_dummies(table[col])], axis=1)
    return pd.concat([table.drop(columns=cat_feature), dummy], axis=1)

In [5]:
whole_dummy = dummy_rep(whole_set, whole_set.columns[cat_feature])
train_dummy = whole_dummy.iloc[0:train_set.shape[0]]
test_dummy = whole_dummy.iloc[train_set.shape[0]:]

In [6]:
scaler = StandardScaler()
train_dummy_std = scaler.fit_transform(train_dummy)
test_dummy_std = scaler.transform(test_dummy)

In [50]:
train_dummy_std.shape

(297020, 382)

In [69]:
input_layer = Input(shape=(382, ))

# encoder
#encoded = Dense(256, activation="relu")(input_layer)
#encoded = Dropout(0.8)(encoded)
encoded = Dense(128, activation="relu")(input_layer)
#encoded = Dropout(0.8)(encoded)
encoded = Dense(64, activation="relu")(encoded)
encoded = Dense(32, activation="relu")(encoded)

#decoder
#decoded = Dense(32, activation="relu")(encoded)
decoded = Dense(64, activation="relu")(decoded)
decoded = Dense(128, activation="relu")(decoded)
#encoded = Dropout(0.8)(decoded)
#encoded = Dense(256, activation="relu")(decoded)
#encoded = Dropout(0.8)(decoded)
output_layer = Dense(382, activation="sigmoid")(decoded)

autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adam", loss="mse")

In [70]:
autoencoder.fit(train_dummy_std, train_dummy_std,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(test_dummy_std, test_dummy_std))

Train on 297020 samples, validate on 99895 samples
Epoch 1/50
297020/297020 [==============================] - 19s 64us/step - loss: 0.9347 - val_loss: 8.7272
Epoch 2/50
297020/297020 [==============================] - 18s 60us/step - loss: 0.9050 - val_loss: 8.7183
Epoch 3/50
297020/297020 [==============================] - 17s 58us/step - loss: 0.9010 - val_loss: 8.7170
Epoch 4/50
297020/297020 [==============================] - 17s 57us/step - loss: 0.8977 - val_loss: 8.7120
Epoch 5/50
297020/297020 [==============================] - 17s 57us/step - loss: 0.8907 - val_loss: 8.7055
Epoch 6/50
297020/297020 [==============================] - 17s 57us/step - loss: 0.8859 - val_loss: 8.7039
Epoch 7/50
297020/297020 [==============================] - 18s 60us/step - loss: 0.8831 - val_loss: 8.7011
Epoch 8/50
297020/297020 [==============================] - 17s 57us/step - loss: 0.8801 - val_loss: 8.6994
Epoch 9/50
297020/297020 [==============================] - 17s 57us/step - loss: 0.8

KeyboardInterrupt: 

In [17]:
autoencoder = VanillaAutoencoder(n_feat=train_dummy_std.shape[1],
                                 n_epoch=100,
                                 batch_size=250,
                                 encoder_layers=3,
                                 decoder_layers=3,
                                 n_hidden_units=100,
                                 encoding_dim=50,
                                 denoising=None)

In [18]:
autoencoder.fit(train_dummy_std, y_train)

Train on 207914 samples, validate on 89106 samples
Epoch 1/100
207914/207914 [==============================] - 27s 132us/step - loss: 0.8960 - val_loss: 1.0644
Epoch 2/100
207914/207914 [==============================] - 26s 124us/step - loss: 0.8359 - val_loss: 1.0528
Epoch 3/100
207914/207914 [==============================] - 27s 129us/step - loss: 0.8285 - val_loss: 1.0597
Epoch 4/100
207914/207914 [==============================] - 26s 124us/step - loss: 0.8247 - val_loss: 1.0879
Epoch 5/100
207914/207914 [==============================] - 25s 118us/step - loss: 0.8224 - val_loss: 1.0771
Epoch 6/100
207914/207914 [==============================] - 23s 110us/step - loss: 0.8208 - val_loss: 1.0751
Epoch 7/100
207914/207914 [==============================] - 22s 108us/step - loss: 0.8194 - val_loss: 1.0307
Epoch 8/100
207914/207914 [==============================] - 24s 115us/step - loss: 0.8184 - val_loss: 1.0302
Epoch 9/100
207914/207914 [==============================] - 25s 121u

VanillaAutoencoder(batch_size=250, decoder_layers=3, denoising=None,
          encoder_layers=3, encoding_dim=50, n_epoch=100, n_feat=382,
          n_hidden_units=100)

In [19]:
train_ae_feature = autoencoder.transform(train_dummy_std)
train_ae_feature = pd.DataFrame(train_ae_feature, columns=["ae_feat_" + str(i+1) for i in range(50)])
test_ae_feature = autoencoder.transform(test_dummy_std)
test_ae_feature = pd.DataFrame(test_ae_feature, columns=["ae_feat_" + str(i+1) for i in range(50)])

In [32]:
train_ae_feature["order_id"] = train_set["order_id"]
train_ae_feature = train_ae_feature[["order_id"] + list(train_ae_feature.columns[:-1])]
test_ae_feature["order_id"] = test_set["order_id"]
test_ae_feature = test_ae_feature[["order_id"] + list(test_ae_feature.columns[:-1])]

In [34]:
train_ae_feature.to_csv("train_ae_feature.csv", encoding="utf-8", index=False)
test_ae_feature.to_csv("test_ae_feature.csv", encoding="utf-8", index=False)